In [ ]:
!pip install --upgrade openai
!pip install xlrd
!pip install -U sentence-transformers

In [ ]:
import numpy as np
dataset = np.load('/content/SP-train.npy', allow_pickle=True)
answers = []
questions = []
choices = []
for i in range(len(dataset)):
  answers.append(dataset[i]["answer"])
  questions.append(dataset[i]["question"])
  choices.append(dataset[i]["choice_list"])

In [ ]:
from openai import OpenAI
import math
from numpy.linalg import norm

client = OpenAI(api_key='API_KEY')
import openpyxl
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

wb = openpyxl.load_workbook('/content/EncodedQuestions.xlsx')
ws = wb.active

def findRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim > highest:
        highest = sim
        nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]
def farRiddle(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  lowest = 1
  for i in range(1,ws.max_row):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim < lowest:
        lowest = sim
        nearest = i
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value]
def topTwoRiddles(Riddle):
  question_emb = model.encode(Riddle)
  A= question_emb
  norm_A = norm(A)
  highest = 0
  secondhighest = 0
  secondnearest = 0
  for i in range(1,ws.max_row+1):
    B = [float(x) for x in ws["C"+str(i)].value.split(",")]
    sim = np.dot(A,B)/(norm_A*norm(B))
    if sim > highest:
        highest = sim
        nearest = i
    elif sim>secondhighest:
        secondhighest = sim
        secondnearest = i
  print(highest, nearest)
  return [ws["A"+str(nearest)].value, ws["B"+str(nearest)].value, ws["A"+str(secondnearest)].value, ws["B"+str(secondnearest)].value]


In [ ]:
#Check Baseline
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    demonstration = "Please pick the best choice for the brain teaser. Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo-1106",
      messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt}
      ]
    )
    pred = completion.choices[0].message.content
    answer = ""
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    #Calculate Accuracy
    if answer  == answers[x]:
      correct += 1
      print("Correct" + pred)
    else:
      print("Incorrect:" + pred)
    count += 1

print(correct, count, correct/count)

In [ ]:
#Check Baseline + Task Description
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    demonstration = "Please pick the best choice for the sentence puzzle brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt}
      ]
    )
    pred = completion.choices[0].message.content
    answer = ""
    val = similar(choices[x][0], pred)
    if val < similar(choices[x][1], pred):
      answer = choices[x][1]
      val = similar(choices[x][1],pred)
    if val < similar(choices[x][2], pred):
      answer = choices[x][2]
      val = similar(choices[x][2],pred)
    if val < similar(choices[x][3], pred):
      answer = choices[x][3]
      val = similar(choices[x][3],pred)
    #Calculate Accuracy
    if answer  == answers[x]:
      correct += 1
      print("Correct" + pred)
    else:
      print("Incorrect:" + pred)
    count += 1

print(correct, count, correct/count)

In [ ]:
#Check Baseline + Task Description + Explain
from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    demonstration = "Please pick the best choice for the sentence puzzle brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": 'You are a riddle solving expert that explains solutions step by step and responds in raw JSON format following the format { "response": <answer>, "explanation": <explaination> }.'},
        {"role": "user", "content": prompt}
      ]
    )
    #print(completion['choices'][0]['message']['content'])
    pred = completion.choices[0].message.content
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar("A", pred)
    if val < similar("B", pred):
      answer = choices[x][1]
      val = similar("B",pred)
    if val < similar("C", pred):
      answer = choices[x][2]
      val = similar("C",pred)
    if val < similar("D", pred):
      answer = choices[x][3]
      val = similar("D",pred)
    #Calculate Accuracy
    print(answer)
    if answer  == answers[x]:
      correct += 1
      print("Correct" + answer)
      print(answers[x])
    else:
      print("Incorrect:" + pred)
      print(answers[x])

    count += 1

print(correct, count, correct/count)

In [ ]:
#Oneshot

from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    closeRiddle = findRiddle(questions[x])
    #closeRiddle = farRiddle(questions[x])
    demonstration = "Please pick the best choice for the sentence puzzle brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Questions with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": 'You are a helpful assistant.'},
        {"role": "user", "content": prompt}
      ]
    )
    #print(completion['choices'][0]['message']['content'])
    pred = completion.choices[0].message.content
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar("A", pred)
    if val < similar("B", pred):
      answer = choices[x][1]
      val = similar("B",pred)
    if val < similar("C", pred):
      answer = choices[x][2]
      val = similar("C",pred)
    if val < similar("D", pred):
      answer = choices[x][3]
      val = similar("D",pred)
    #Calculate Accuracy
    print(answer)
    if answer  == answers[x]:
      correct += 1
      print("Correct" + answer)
      print(answers[x])
    else:
      print("Incorrect:" + pred)
      print(answers[x])

    count += 1

print(correct, count, correct/count)

In [ ]:
#Oneshot Explain
from difflib import SequenceMatcher
import json
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

correct = 0
count = 0
for x in range(0,len(questions)):
    #closeRiddle = findRiddle(questions[x])
    closeRiddle = farRiddle(questions[x])
    demonstration = "Please pick the best choice for the sentence puzzle brain teaser (Sentence-type brain teaser where the puzzle defying commonsense is centered on sentence snippets). Each brain teaser has only one possible solution including the choice none of above, answer should only provide the choice:"
    prompt = demonstration + "Question:" + questions[x]+ " \n Choice:" + "\n (A) " + choices[x][0] + "\n (B) " + choices[x][1] + "\n (C) " + choices[x][2] + "\n (D) " +choices[x][3] + "\n Example Questions with corresponding example answer:" +"\n" +closeRiddle[0] + "\n The answer would be" + closeRiddle[1]
    completion = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
        {"role": "system", "content": 'You are a riddle solving expert that explains solutions step by step and responds in raw JSON format following the format { "response": <answer>, "explanation": <explaination> }.'},
        {"role": "user", "content": prompt}
      ]
    )
    #print(completion['choices'][0]['message']['content'])
    pred = completion.choices[0].message.content
    try:
      pred = json.loads(pred)['response']
    except:
      pred = pred
    answer = choices[x][0]
    val = similar("A", pred)
    if val < similar("B", pred):
      answer = choices[x][1]
      val = similar("B",pred)
    if val < similar("C", pred):
      answer = choices[x][2]
      val = similar("C",pred)
    if val < similar("D", pred):
      answer = choices[x][3]
      val = similar("D",pred)
    #Calculate Accuracy
    print(answer)
    if answer  == answers[x]:
      correct += 1
      print("Correct" + answer)
      print(answers[x])
    else:
      print("Incorrect:" + pred)
      print(answers[x])

    count += 1

print(correct, count, correct/count)